In [51]:
# from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import geopandas as gpd
import rasterio as rio
from pathlib import Path
from shapely.geometry import Point
import pyproj
from tqdm.auto import tqdm
from rasterio import Affine
from shapely.geometry import box
from shapely.ops import transform
from pyproj import Transformer
from multiprocess import Pool
import numpy as np
from math import sqrt
import time
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [52]:
# this is the world tide api key, let me know if you need it
planetary_computer_key_file = Path.cwd() / "planitary_computer_key.txt"
planetary_computer_api_key = planetary_computer_key_file.read_text().strip()
os.environ["PC_SDK_SUBSCRIPTION_KEY"] = planetary_computer_api_key

In [109]:
source = "OSM"
# source = "Aus"
# source = "NZ"

In [110]:
data_dir = Path(
    "/Users/Nick/Library/Mobile Documents/com~apple~CloudDocs/QGIS/Coastline training data v2/"
)
data_dir.exists()

True

In [111]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [112]:
vector_points_path = data_dir / f"{source}/{source} training data.gpkg"
print(vector_points_path.exists())
vector_points = gpd.read_file(vector_points_path)

True


In [113]:
ocean_path = data_dir / f"{source}/{source} polygons.gpkg"
print(ocean_path.exists())
coastline_path = data_dir / f"{source}/{source} lines.gpkg"
print(coastline_path.exists())
prefix = f"{source}_80"

True
True


In [114]:
images_path = Path.cwd() / "training data" / "images_2_3_4_8_V3"
sar_path = Path.cwd() / "training data" / "SAR_V3"
labels_path = Path.cwd() / "training data" / "labels_2_3_4_8_V3"
labels_path.mkdir(exist_ok=True, parents=True)
images_path.mkdir(exist_ok=True, parents=True)
sar_path.mkdir(exist_ok=True, parents=True)

In [115]:
bands = ["B02", "B03", "B04", "B08"]
time_steps = 6

In [116]:
def find_prods(point, time_of_interest="2022-01-01/2023-01-01"):
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=point,
        datetime=time_of_interest,
        query={"eo:cloud_cover": {"lt": 80}},
    )
    items = search.item_collection()
    return items

In [117]:
def wgs_point_to_local_box(product, wgs_point, vector_data_crs_number):
    local_crs_number = product.properties["proj:epsg"]
    source_crs = pyproj.CRS(f"EPSG:{vector_data_crs_number}")
    target_crs = pyproj.CRS(f"EPSG:{local_crs_number}")
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    x, y = wgs_point.x, wgs_point.y
    x_transformed, y_transformed = transformer.transform(x, y)
    point_utm = Point(x_transformed, y_transformed)
    point_buffer = point_utm.buffer(2560 / 2)
    return point_buffer.bounds

In [118]:
# get epsg code
vector_data_crs_number = vector_points.crs.to_epsg()
vector_data_crs_number

4326

In [119]:
total_bands = time_steps * len(bands)
total_bands

24

In [120]:
# for row in tqdm(vector_points.iterrows(), total=len(vector_points)):
def downlaod_image(row):
    try:
        wgs_point = row[1].geometry

        if wgs_point is None:
            return

        export_name = f"{row[0]}_{prefix}.tif"
        export_path = images_path / export_name

        if export_path.stem in skip_list:
            return

        if export_path.exists():
            return

        search_point = {"type": "Point", "coordinates": [wgs_point.x, wgs_point.y]}

        products = find_prods(search_point)
        arrays = []
        # print(products)
        for product in products:
            # print(product.id)
            b_box = wgs_point_to_local_box(product, wgs_point, vector_data_crs_number)
            band_arrays = []
            skip = False
            for band in bands:
                if skip:
                    continue
                # time.sleep(2)
                with rio.open(product.assets[band].href) as src:
                    window = rio.windows.from_bounds(*b_box, src.transform)

                    array = src.read(1, window=window)
                    if array.shape != (256, 256):
                        print(f"Array shape is {array.shape} for {export_name}")
                        skip = True
                        continue
                    transform = rio.windows.transform(window, transform=src.transform)
                    profile = src.profile.copy()
                    if band == bands[0]:
                        # print(np.count_nonzero(array == 0))
                        if np.count_nonzero(array == 0) > 100:
                            skip = True
                            continue

                    band_arrays.append(array)

            for band_array in band_arrays:
                arrays.append(band_array)

            if len(arrays) == total_bands:
                break

        if len(arrays) != total_bands:
            print(f"Could not find 3 images for {export_name}")
            time.sleep(2)
            return
        profile.update(
            {
                "height": array.shape[0],
                "width": array.shape[1],
                "transform": transform,
                "count": total_bands,
            }
        )
        with rio.open(export_path, "w", **profile) as dst:
            dst.write(np.array(arrays))
    except Exception as e:
        print(e)
        print(f"Failed to download {export_name}")
        time.sleep(2)
        return

In [121]:
vector_points = vector_points.iloc[::-1]

In [77]:
skip_list = [
    "614_OSM_80",
    "611_OSM_80",
    "579_OSM_80",
    "577_OSM_80",
    "554_OSM_80",
    "550_OSM_80",
    "536_OSM_80",
    "533_OSM_80",
    "531_OSM_80",
    "526_OSM_80",
    "523_OSM_80",
    "509_OSM_80",
    "490_OSM_80",
    "478_OSM_80",
    "466_OSM_80",
    "465_OSM_80",
    "462_OSM_80",
    "797_OSM_80",
    "798_OSM_80",
    "791_OSM_80",
    "777_OSM_80",
    "775_OSM_80",
    "758_OSM_80",
    "716_OSM_80",
    "706_OSM_80",
    "691_OSM_80",
    "325_OSM_80",
    "355_OSM_80",
]

In [122]:
def worker(i):
    if i[0] not in skip_list:
        downlaod_image(i)


# Number of threads you want to run in parallel
num_threads = 8

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    list(tqdm(executor.map(worker, vector_points.iterrows()), total=len(vector_points)))

100%|██████████| 800/800 [00:39<00:00, 20.27it/s]


In [89]:
# for i in tqdm(vector_points.iterrows(), total=len(vector_points)):
#     if i[0] not in skip_list:
#         downlaod_image(i)

 62%|██████▏   | 56/90 [00:04<00:02, 12.15it/s]


KeyboardInterrupt: 

In [105]:
stac_client = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

In [106]:
def search_sar(point):
    search = stac_client.search(
        collections=["sentinel-1-rtc"],
        intersects=point,
    )
    items = search.get_all_items()
    return items

In [107]:
def downlaod_s1_rtc(row):
    try:
        wgs_point = row[1].geometry

        if wgs_point is None:
            return

        export_name = f"{row[0]}_{prefix}.tif"
        export_path = sar_path / export_name

        if export_path.exists():
            return

        search_point = {"type": "Point", "coordinates": [wgs_point.x, wgs_point.y]}

        products = search_sar(search_point)

        skip = False
        for product in products:
            if not skip:
                product = planetary_computer.sign(product)
                b_box = wgs_point_to_local_box(
                    product, wgs_point, vector_data_crs_number
                )
                with rio.open(product.assets["vh"].href) as src:
                    window = rio.windows.from_bounds(*b_box, src.transform)

                    array = src.read(1, window=window)

                    transform = rio.windows.transform(window, transform=src.transform)
                    profile = src.profile.copy()
                    if array.shape == (256, 256):
                        if np.count_nonzero(array == 0) < 100:
                            skip = True
        if skip:
            profile.update(
                {
                    "height": array.shape[0],
                    "width": array.shape[1],
                    "transform": transform,
                    "count": 1,
                }
            )
            with rio.open(export_path, "w", **profile) as dst:
                dst.write(np.array(array), 1)
    except Exception as e:
        print(e)
        print(f"Failed to download {export_name}")
        time.sleep(2)
        return

In [108]:
def worker(i):
    if i[0] not in skip_list:
        downlaod_s1_rtc(i)


# Number of threads you want to run in parallel
num_threads = 8

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    list(tqdm(executor.map(worker, vector_points.iterrows()), total=len(vector_points)))

  0%|          | 0/900 [00:00<?, ?it/s]/usr/local/Caskroom/miniforge/base/envs/gis_311/lib/python3.11/site-packages/pystac_client/item_search.py:841: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(
  2%|▏         | 20/900 [00:18<09:34,  1.53it/s] 

'vh'
Failed to download 20_Aus_80.tif
'vh'
Failed to download 23_Aus_80.tif
'vh'
Failed to download 22_Aus_80.tif
'vh'
Failed to download 21_Aus_80.tif


 22%|██▏       | 199/900 [01:39<05:28,  2.13it/s]

'vh'
Failed to download 203_Aus_80.tif


 26%|██▌       | 231/900 [01:52<04:06,  2.72it/s]

'vh'
Failed to download 233_Aus_80.tif
'vh'
Failed to download 234_Aus_80.tif


 71%|███████   | 641/900 [04:47<01:44,  2.48it/s]

'vh'
Failed to download 644_Aus_80.tif


100%|██████████| 900/900 [06:52<00:00,  2.18it/s]


In [136]:
def rasterise(args):
    if args is None:
        return
    (
        label_export_path,
        coastline_lines_clipped,
        clipped_gdf,
        profile,
        array_transform,
        max_dist,
    ) = args
    if label_export_path.exists():
        return
    clipped_gdf["geometry"] = clipped_gdf["geometry"].buffer(0)
    array = np.zeros((profile["height"], profile["width"]), dtype=np.float32)
    for row in range(profile["height"]):
        for col in range(profile["width"]):
            x, y = array_transform * (col + 0.5, row + 0.5)
            point = Point(x, y)

            if len(coastline_lines_clipped) == 0:
                min_distance = max_dist

            else:
                min_distance = min(
                    geom.distance(point) for geom in coastline_lines_clipped["geometry"]
                )

            if min_distance > max_dist:
                min_distance = max_dist
            min_distance = sqrt(min_distance)
            if clipped_gdf.intersects(point).any():
                array[row, col] = -min_distance
            else:
                array[row, col] = min_distance

    profile.update({"count": 1, "dtype": "float32", "compress": "lzw"})

    with rio.open(label_export_path, "w", **profile) as dst:
        dst.write(array, 1)

In [134]:
def make_label(image, coastline_lines, ocean_polygons, max_dist=50):
    try:
        label_export_path = labels_path / image.name

        if label_export_path.exists():
            return

        with rio.open(image) as src:
            pixel_size = src.transform[0]
            extent = src.bounds
            raster_poly = box(*extent)
            raster_crs = src.crs
            profile = src.profile.copy()

        transformer = Transformer.from_crs(
            raster_crs, ocean_polygons.crs, always_xy=True
        )
        reprojected_polygon = transform(transformer.transform, raster_poly)

        clipped_gdf = (
            gpd.clip(ocean_polygons, reprojected_polygon).dissolve().to_crs(raster_crs)
        )

        coastline_lines_clipped = gpd.clip(coastline_lines, reprojected_polygon).to_crs(
            raster_crs
        )

        minx, miny, maxx, maxy = extent
        array_transform = Affine.translation(minx, maxy) * Affine.scale(
            pixel_size, -pixel_size
        )
        args = (
            label_export_path,
            coastline_lines_clipped,
            clipped_gdf,
            profile,
            array_transform,
            max_dist,
        )
        #
        return args
    except Exception as e:
        print(e)
        print(f"Failed to make label for {image.name}")
        return

In [130]:
images = list(images_path.glob(f"*{prefix}.tif"))
len(images)

783

In [131]:
ocean_polygons = gpd.read_file(ocean_path)

In [132]:
coastline = gpd.read_file(coastline_path)

In [135]:
args_list = []
for image in tqdm(images):
    args_list.append(
        make_label(image, coastline_lines=coastline, ocean_polygons=ocean_polygons)
    )

100%|██████████| 783/783 [02:35<00:00,  5.04it/s]


In [137]:
with Pool() as p:
    list(tqdm(p.imap(rasterise, args_list), total=len(args_list)))

100%|██████████| 783/783 [16:19<00:00,  1.25s/it]  


In [143]:
images = images_path.glob("*.tif")
sars = sar_path.glob("*.tif")
lebels = labels_path.glob("*.tif")

In [144]:
remove_list = []
for image in tqdm(images):
    expected_label = labels_path / image.name
    expected_sar = sar_path / image.name
    if not expected_label.exists():
        print(f"Missing label for {image.name}")
        remove_list.append(image)
    if not expected_sar.exists():
        print(f"Missing SAR for {image.name}")
        # remove_list.append(image)

1712it [00:00, 10923.80it/s]

Missing SAR for 563_OSM_80.tif
Missing SAR for 440_OSM_80.tif
Missing SAR for 415_OSM_80.tif
Missing SAR for 422_OSM_80.tif
Missing SAR for 548_OSM_80.tif
Missing SAR for 121_OSM_80.tif
Missing SAR for 28_OSM_80.tif
Missing SAR for 409_OSM_80.tif
Missing SAR for 20_Aus_80.tif
Missing SAR for 418_OSM_80.tif
Missing SAR for 130_OSM_80.tif
Missing SAR for 652_OSM_80.tif
Missing SAR for 559_OSM_80.tif
Missing SAR for 433_OSM_80.tif
Missing SAR for 25_OSM_80.tif
Missing SAR for 451_OSM_80.tif
Missing SAR for 545_OSM_80.tif
Missing SAR for 210_OSM_80.tif
Missing SAR for 657_OSM_80.tif
Missing SAR for 660_OSM_80.tif
Missing SAR for 448_OSM_80.tif
Missing SAR for 209_OSM_80.tif
Missing SAR for 129_OSM_80.tif
Missing SAR for 436_OSM_80.tif
Missing SAR for 445_OSM_80.tif
Missing SAR for 551_OSM_80.tif
Missing SAR for 427_OSM_80.tif
Missing SAR for 31_OSM_80.tif
Missing SAR for 410_OSM_80.tif
Missing label for 831_Aus_80.tif
Missing SAR for 676_OSM_80.tif
Missing SAR for 195_OSM_80.tif
Missing SA

In [146]:
len(remove_list)

1

In [147]:
for i in remove_list:
    os.remove(i)